In [41]:
!cp out-tinystories/new_model.eqx out-tinystories/model.eqx

In [46]:
!python train.py config/train_new_tinystories.py --out_path="out-tinystories/model.eqx" --max_iters=250 

Overriding config with config/train_new_tinystories.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_path = 'out-tinystories/new_model.eqx'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 5 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
tensorboard_log = True # override via command line if you like
log_project = "exp3"
log_run_name = "train_new_tokenzier"  # 'run' + str(time.time())

dataset = 'new_tinystories'
gradient_accumulation_steps = 1
batch_size = 32
block_size = 100 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 16
n_head = 12
n_embd = 512
dropout = 0.0

learning_rate = 1e-2 # with baby networks can afford to go a bit higher
max_iters = 500
lr_decay_iters = 500 # make equal to max_iters usually
mi

In [ ]:
import jax
import equinox as eqx


emb = eqx.nn.Embedding(3, 4, key=jax.random.key(1))
x = jax.numpy.array([[1, 0, 0.0],[0,1,0]])

print(emb.weight.shape)
print(jax.numpy.matmul(x, emb.weight))
print(emb.weight[0])
print(emb.weight[1])
print(emb.weight[2])

In [ ]:
import jax
import jax.numpy as jnp

# @eqx.filter_jit
def sparsemax(input: jax.Array, marg):
    arr = jnp.sort(input, descending=True)
    y = marg + jnp.arange(1, arr.shape[0] + 1) * arr
    sum = jnp.cumsum(arr)
    k_z = (
        jnp.count_nonzero(jnp.where(jnp.greater(y - sum, 0), y - sum, 0)) - 1
    )  # Makes sure that if two elements are equal we take the latter.

    thresh = (sum[k_z] - marg) / (k_z + 1)
    return jnp.where(jnp.greater(input - thresh, 0), input - thresh, 0)

# @eqx.filter_jit
def sinkhorn(X, mu, nu):
    P = jnp.ones_like(X) / mu.shape[0]  # Divide by v
    Q = jnp.zeros_like(X)
    for _ in range(0,1):
        Y = jax.vmap(sparsemax, in_axes=(0, 0))(X + P, mu)
        P = X + P - Y
        X = jnp.transpose(
            jax.vmap(sparsemax, in_axes=(0, 0))(jnp.transpose(Y + Q), nu)
        )
        Q = Y + Q - X
    return X

(u, v) = (2, 12)

X = jax.random.normal(jax.random.key(1), shape=(v, u))
mu = jnp.ones(shape=(v,)) /v
nu = jnp.ones(shape=(u,)) / u

pouet = jnp.array([1, 3, 2], dtype=jnp.float32)
pouet_voulu = []


def loss(x, y):
    map = sinkhorn(X, mu, nu)[x]
    
    return jnp.mean((map - y) ** 2)

grad = jax.grad(loss)(pouet, jnp.zeros(shape=(2, 3)))


ValueError: Incompatible shapes for broadcasting: shapes=[(12, 2), (3,)]

In [57]:
print(map)

[[0.06250003 0.01388888]
 [0.06250003 0.01388888]
 [0.         0.09722225]]
